<a href="https://colab.research.google.com/github/Ehtisham1053/Natural-Language-Processing/blob/main/Skip_grams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skip-gram Model:
The Skip-gram model is a technique used in Word2Vec to learn word embeddings. Unlike CBOW(Continuous Bag of Words), which predicts the target word from context words, Skip-gram predicts context words from the target word.



---



---



# 📌 How Skip-gram Works
1. Training Data: A large corpus of text (e.g., Wikipedi
articles).

2. Word Pairs: For each word, it predicts the surrounding words (context).

3. Window Size: Defines how many words before and after the target word are considered.

4. Neural Network: Uses a simple shallow neural network to learn word vectors.

5. Optimization: Uses negative sampling or hierarchical softmax to improve efficiency.

6. Output: A dense vector representation for each word.



In [4]:
!pip install --no-cache-dir --force-reinstall gensim nltk matplotlib wikipedia-api

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 205.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 182.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 180.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 199.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 167.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 199.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 195.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 166.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 173.4 MB/s eta 0:00:

In [11]:
import gensim
from gensim.models import Word2Vec
import wikipediaapi
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import matplotlib.pyplot as plt
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='MyProjectName/1.0 (MyProjectURL; MyEmail@example.com)'
)


topics = ["Machine Learning", "Artificial Intelligence", "Deep Learning", "Neural Networks",
          "Natural Language Processing", "Big Data", "Computer Vision", "Data Science"]

wiki_texts = []
for topic in topics:
    page = wiki.page(topic)
    if page.exists():
        wiki_texts.append(page.text)

large_wiki_text = " ".join(wiki_texts)

print("Sample Wikipedia Text (First 500 characters):\n", large_wiki_text[:500])

Sample Wikipedia Text (First 500 characters):
 Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML finds application in many fields, 


In [12]:
sentences = nltk.sent_tokenize(large_wiki_text)
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

stop_words = set(stopwords.words("english"))
tokenized_sentences = [[word for word in sentence if word not in stop_words and word not in string.punctuation]
                       for sentence in tokenized_sentences]

print("Tokenized Sentences:\n", tokenized_sentences)



Tokenized Sentences:
 [['machine', 'learning', 'ml', 'field', 'study', 'artificial', 'intelligence', 'concerned', 'development', 'study', 'statistical', 'algorithms', 'learn', 'data', 'generalize', 'unseen', 'data', 'thus', 'perform', 'tasks', 'without', 'explicit', 'instructions'], ['within', 'subdiscipline', 'machine', 'learning', 'advances', 'field', 'deep', 'learning', 'allowed', 'neural', 'networks', 'class', 'statistical', 'algorithms', 'surpass', 'many', 'previous', 'machine', 'learning', 'approaches', 'performance'], ['ml', 'finds', 'application', 'many', 'fields', 'including', 'natural', 'language', 'processing', 'computer', 'vision', 'speech', 'recognition', 'email', 'filtering', 'agriculture', 'medicine'], ['application', 'ml', 'business', 'problems', 'known', 'predictive', 'analytics'], ['statistics', 'mathematical', 'optimization', 'mathematical', 'programming', 'methods', 'comprise', 'foundations', 'machine', 'learning'], ['data', 'mining', 'related', 'field', 'study', 'f

In [14]:
skipgram_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=100,  # Embedding size
                          window=5,        # Context window size
                          sg=1,            # sg=1 means Skip-gram, sg=0 means CBOW
                          min_count=3,     # Ignore words with frequency <3
                          workers=4)       # Number of CPU threads

skipgram_model.save("skipgram_wikipedia.model")

print("Skip-gram Model Training Completed!")


Skip-gram Model Training Completed!


### find similar words

In [15]:
model = Word2Vec.load("skipgram_wikipedia.model")

print("\nWords similar to 'data':")
for word, similarity in model.wv.most_similar("data", topn=5):
    print(f"{word}: {similarity:.4f}")

print("\nWords similar to 'intelligence':")
for word, similarity in model.wv.most_similar("intelligence", topn=5):
    print(f"{word}: {similarity:.4f}")



Words similar to 'data':
big: 0.9931
analysis: 0.9916
science: 0.9909
sets: 0.9904
size: 0.9894

Words similar to 'intelligence':
called: 0.9867
neurons: 0.9859
use: 0.9853
solve: 0.9852
general: 0.9850
